## Preprocessing

### Import

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from local_library.image_operations import *
from local_library.ellipsoid import *
from sklearn.mixture import GaussianMixture

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import time

### Configuration

In [ ]:

# Path to the folder containing seedling images
IMAGE_FOLDER_PATH = "D:/#Work/4-1 WiL/Workspace/Dataset/Dataset1/RedCos/"
# IMAGE_FOLDER_PATH = "D:/#Work/4-1 WiL/Workspace/Dataset/Dataset1/RedCos - Copy/"
# IMAGE_FOLDER_PATH = "D:/#Work/4-1 WiL/Workspace/Dataset/Dataset1/Butterhead/"
# IMAGE_FOLDER_PATH = "D:/#Work/4-1 WiL/Workspace/Dataset/Dataset1/RedButterhead/"
IMAGE_FILE_FORMAT = ".jpg"  # Image file format

# Tray dimensions and structure
TRAY_ROW_COUNT = 8                  # Number of rows in the seedling tray
TRAY_COLUMN_COUNT = 16              # Number of columns in the seedling tray
TRAY_HEIGHT = 290                   # Tray height in millimeters
TRAY_WIDTH = 580                    # Tray width in millimeters



# Number of Gaussian models used for probability calculations and Gaussian Mixture Model (GMM) 
MIN_NUM_GAUSSIAN_MODELS = 3         # Minimum number of Gaussian models per cell
MAX_NUM_GAUSSIAN_MODELS = 5         # Maximum number of Gaussian models per cell

# Image shape definitions based on tray dimensions
IMAGE_HEIGHT = TRAY_ROW_COUNT * 30 
IMAGE_WIDTH = TRAY_COLUMN_COUNT * 30
IMAGE_SHAPE = np.array([IMAGE_HEIGHT, IMAGE_WIDTH])
CELL_HEIGHT, CELL_WIDTH = IMAGE_HEIGHT // TRAY_ROW_COUNT, IMAGE_WIDTH // TRAY_COLUMN_COUNT
CELL_SHAPE = np.array([CELL_HEIGHT, CELL_WIDTH])
HALF_CELL_HEIGHT, HALF_CELL_WIDTH = CELL_HEIGHT // 2, CELL_WIDTH // 2
HALF_CELL_SHAPE = CELL_SHAPE // 2
PIXELS_TO_REAL_DISTANCE_RATIO = TRAY_WIDTH / IMAGE_WIDTH

# Initial confidence values and scaling factors used in Bayesian segmentation
INIT_CONFIDENCE = 0.1                           # Minimum initial confidence for seedling identification
SPATIAL_BACKGROUND_PRIOR_SCALING = 1.5          # Scaling factor to prevent division by zero in Bayesian segmentation

# Threshold values for seedling probability and model updates
MEAN_SEEDLING_COLOR_CONFIDENCE_THRESHOLD = 0.01             # Threshold for color confidence; exceeding this means reliable seedling color
MIN_SEEDLING_PIXEL_IN_IMAGE_FOR_CALCULATION = 10            # Minimum number of seedling pixels required for calculation
SEEDLING_SIZE_TO_CELL_RATIO_THRESHOLD = 0.01                # Threshold ratio for seedling size relative to cell size
PROBABILITY_THRESHOLD_FOR_UPDATE_COLOR_MODEL = 0.1          # Probability threshold for updating the color model

# Covariance scaling factors for prior probability in the spatial domain
INIT_COVARIANCE_PRIOR_SCALING = 0.025                               # Covariance scaling factor for initial seedling position
COVARIANCE_POSITION_UPDATE_COLOR_CONFIDENCE_SCALING = 1             # Covariance scaling factor for position weight in color confidence updating
COVARIANCE_PRIOR_ALL_SEEDLING_CELL_SCALING = 0.15                   # Covariance scaling factor for all seedling cells
COVARIANCE_PRIOR_EMPTY_CELL_SCALING = 0.002                         # Covariance scaling factor for empty cells
MAX_COVARIANCE_PRIOR_EACH_SEEDLING_CELL_SCALING = 0.007             # Maximum allowed scaling for covariance of seedling cells
MIN_COVARIANCE_PRIOR_EACH_SEEDLING_CELL_SCALING = 0.001            # Minimum allowed scaling for covariance of seedling cells
COVARIANCE_PRIOR_EACH_SEEDLING_CELL_SCALING_RATIO = 1.5             # Scaling ratio for adjusting seedling cell covariance

# Weights and learning rates for updating seedling status confidence
BACKGROUND_CONFIDENCE_WEIGHT = 0.99                         # Weight for background confidence updates
PROBABILITY_UPDATE_COLOR_CONFIDENCE_WEIGHT = 0.8            # Weight for updating seedling color confidence based on probability
POSITION_UPDATE_COLOR_CONFIDENCE_WEIGHT = 0.6               # Weight for updating seedling color confidence based on seedling position
STATUS_UPDATE_STATUS_CONFIDENCE_WEIGHT = 0.8                # Weight for updating seedling status confidence based on seedling status
COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT = 0.7                 # Weight for updating seedling status confidence based on seedling color model
POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT = 0.6              # Weight for updating seedling status confidence based on seedling position


SEEDLING_STATUS_CONFIDENCE_THRESHOLD = 0.9


### Import image

In [ ]:
image_names_list = os.listdir(IMAGE_FOLDER_PATH)
image_names_list = [image_name.split('.')[0] for image_name in image_names_list]
rgb_images_list = [plt.imread(IMAGE_FOLDER_PATH + image_name + IMAGE_FILE_FORMAT) / 255 for image_name in image_names_list]
rgb_images_list = [resizeImage(image_rgb, 1200, 2400) for image_rgb in rgb_images_list]
resized_rgb_images_list = [resizeImage(image_rgb, IMAGE_HEIGHT, IMAGE_WIDTH) for image_rgb in rgb_images_list]
ycbcr_images_list = [convertRGBtoYCbCr(image) for image in resized_rgb_images_list]
ycbcr_vectors_list = [ image.reshape(-1, image.shape[-1]) for image in ycbcr_images_list]
number_of_images = len(image_names_list)

### Declare variables for storing data

In [ ]:
# Initialize arrays to hold various data for each image in the dataset

# Array to store the status of seedlings in each cell for each image
seedling_status_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT))

# Arrays to store mean positions and covariance matrix of seedlings for each cell in each image
seedling_position_model_means_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT, 2))
seedling_position_model_covariances_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT, 2, 2))

# Array to store the number of Gaussian models used for each image
number_of_color_models_buffer = np.zeros(number_of_images, dtype=int)

# Arrays to store means and covariances of color models for each image
color_model_means_buffer = np.zeros((number_of_images, MAX_NUM_GAUSSIAN_MODELS, 3))
color_model_covariances_buffer = np.zeros((number_of_images, MAX_NUM_GAUSSIAN_MODELS, 3, 3))

# Arrays to store means and covariances of seedling color models for each cell in each image
seedling_color_model_each_cell_means_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT, 3))
seedling_color_model_each_cell_covariances_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT, 3, 3))

# Arrays to store ellipse sizes, seedling sizes (sum of probabilities), and size-to-cell ratios for each cell in each image
ellipse_size_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT))
seedling_size_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT))
seedling_size_to_cell_ratio_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT))

# Array to store the Euclidean distance between seedling center and planting cell center for each cell in each image
distance_position_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT))
# Array to store the Euclidean distance between mean Cb-Cr of each seedling and mean Cb-Cr of the entire tray in YCbCr color space for each cell in each image
distance_color_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT))

# Array to store confidence in seedling status for each cell in each image
seedling_status_confidence_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT))

### Visualization

In [ ]:
plt.figure(figsize=(20,20))
for i in range(number_of_images):
    plt.subplot(7,5,1+i), plt.imshow(resized_rgb_images_list[i]), plt.title(image_names_list[i].split(".")[0])
# fig = plt.figure(figsize=(20,10))
# for i in range(number_of_images):
#     ax = fig.add_subplot(7,5, 1 + i, projection="3d")
#     ax.scatter3D(ycbcr_vectors_list[i][:, 2], ycbcr_vectors_list[i][:, 1], ycbcr_vectors_list[i][:, 0], color="green", marker="o", alpha=0.2, s=2)
#     ax.set_title(image_names_list[i])
#     ax.axes.set_xlim3d(left=-0.5, right=0.5)
#     ax.axes.set_ylim3d(bottom=-0.5, top=0.5)
#     ax.axes.set_zlim3d(bottom=0, top=1)
#     ax.set_xlabel("Cr"), ax.set_ylabel("Cb"), ax.set_zlabel("Y")
#     ax.view_init(elev=7, azim=-7)

In [ ]:
fig = plt.figure(figsize=(20,20))
for i in range(number_of_images):
    ax = fig.add_subplot(7,5, 1 + i, projection="3d")
    ax.scatter3D(ycbcr_vectors_list[i][::10, 2], ycbcr_vectors_list[i][::10, 1], ycbcr_vectors_list[i][::10, 0], color="green", marker="o", alpha=0.2, s=2)
    ax.set_title(image_names_list[i])
    ax.axes.set_xlim3d(left=-0.5, right=0.5)
    ax.axes.set_ylim3d(bottom=-0.5, top=0.5)
    ax.axes.set_zlim3d(bottom=0, top=1)
    ax.set_xlabel("Cr"), ax.set_ylabel("Cb"), ax.set_zlabel("Y")
    ax.view_init(elev=7, azim=-7)

## Identifying Seedling Color

### Init background color

In [ ]:
# Day01 = Background
current_image_index = 0

# Extract the vector of background colors for Day 1 from the list of YCbCr vectors
background_color_vector = ycbcr_vectors_list[current_image_index]

# Calculate the mean and covariance matrix for the background color model
init_mean_background_color, init_covariance_background_color = calculateMeanAndCovariance(background_color_vector)

# Calculate Mahalanobis distances for each background color value from the background color model
background_color_mahalanobis_distances = calculateMahalanobis(background_color_vector, init_mean_background_color, init_covariance_background_color)

# Find the maximum Mahalanobis distance among the background color distances
max_background_color_mahalanobis_distance = np.max(background_color_mahalanobis_distances)

print('Mean of background :', init_mean_background_color)
print('Covariance matrix of background :\n', init_covariance_background_color)
print('Max distance of background color :', max_background_color_mahalanobis_distance)

### Update seedling color confidence

In [ ]:
# Initialize variables for updating the confidence that each pixel's color indicates a seedling
seedling_color_confidence = np.zeros(IMAGE_SHAPE)    

# Create position weight for target positions in the spatial domain to find seedling color vectors
position_weight_update_color_confidence = createPrior(COVARIANCE_POSITION_UPDATE_COLOR_CONFIDENCE_SCALING, IMAGE_SHAPE, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)
position_weight_update_color_confidence = (position_weight_update_color_confidence / position_weight_update_color_confidence.max()) * (2 * POSITION_UPDATE_COLOR_CONFIDENCE_WEIGHT - 1) + (1 - POSITION_UPDATE_COLOR_CONFIDENCE_WEIGHT)

# Visualization
fig = plt.figure(figsize=(40,90))

current_image_index += 1
while( True ):
    
    # Extract the image and color vector for the current image
    current_image = ycbcr_images_list[current_image_index]
    current_image_color_vector = ycbcr_vectors_list[current_image_index]

    # Calculate Mahalanobis distances for each current image color value from the background color model
    current_image_mahalanobis_distances = calculateMahalanobis(current_image_color_vector, init_mean_background_color, init_covariance_background_color)

    # Visualization
    # Identify indices of color values in the current image that are significantly different from the background color model
    seedling_indices = np.where(current_image_mahalanobis_distances > max_background_color_mahalanobis_distance)
    # Extract the color vectors corresponding to the identified seedling indices
    seedling_color_vector = current_image_color_vector[seedling_indices]
    # Identify indices of color values in the current image that are significantly different from the background color model
    background_indices = np.where(current_image_mahalanobis_distances < max_background_color_mahalanobis_distance)
    # Extract the color vectors corresponding to the identified seedling indices
    background_color_vector = current_image_color_vector[background_indices]
    plt.subplot(20, 7, 1 + (7*(current_image_index-1))), plt.imshow(resized_rgb_images_list[current_image_index]), plt.title(image_names_list[current_image_index])
    ax = fig.add_subplot(20, 7, 2 + (7*(current_image_index-1)), projection="3d")
    ax.scatter3D(seedling_color_vector[:, 2], seedling_color_vector[:, 1], seedling_color_vector[:, 0], color="green", marker="o", alpha=0.2, s=2)
    ax.scatter3D(background_color_vector[:, 2], background_color_vector[:, 1], background_color_vector[:, 0], color="black", marker="o", alpha=0.2, s=2)
    ax.set_title('scatter plot'), ax.axes.set_xlim3d(left=-0.5, right=0.5), ax.axes.set_ylim3d(bottom=-0.5, top=0.5), ax.axes.set_zlim3d(bottom=0, top=1), ax.set_xlabel("Cr"), ax.set_ylabel("Cb"), ax.set_zlabel("Y"), ax.view_init(elev=7, azim=-7)
    
    seedling_mask = np.where(current_image_mahalanobis_distances.reshape(IMAGE_SHAPE) > max_background_color_mahalanobis_distance, 1, 0)
    seedling_color_probability = np.where(seedling_mask > 0.5, PROBABILITY_UPDATE_COLOR_CONFIDENCE_WEIGHT, 1 - PROBABILITY_UPDATE_COLOR_CONFIDENCE_WEIGHT)
    
    position_weight = np.where(seedling_mask == 1, position_weight_update_color_confidence, 0.5)
    
    seedling_color_confidence = np.where(((seedling_color_confidence < INIT_CONFIDENCE).astype(int) * (seedling_mask == 1).astype(int)), INIT_CONFIDENCE, seedling_color_confidence)
    
    # Update confidence
    # seedling_color_confidence = calculateBayes( seedling_color_confidence * (seedling_mask * PROBABILITY_UPDATE_COLOR_CONFIDENCE_WEIGHT) * position_weight, (1 - seedling_color_confidence * BACKGROUND_CONFIDENCE_WEIGHT) * (1 - seedling_mask * PROBABILITY_UPDATE_COLOR_CONFIDENCE_WEIGHT) * (1 - position_weight) )
    seedling_color_confidence = calculateBayes( seedling_color_confidence * seedling_color_probability * position_weight, (1 - seedling_color_confidence * BACKGROUND_CONFIDENCE_WEIGHT) * (1 - seedling_color_probability) * (1 - position_weight) )

    #  Calculate mean of seedling color confidence
    mean_seedling_color_confidence = np.mean(seedling_color_confidence)

    # Visualization
    # plt.subplot(20, 7, 3 + (7*(current_image_index-1))), plt.imshow(resized_rgb_images_list[current_image_index] * mask), plt.title('seedling')
    plt.subplot(20, 7, 3 + (7*(current_image_index-1))), plt.imshow(resized_rgb_images_list[current_image_index] * seedling_mask[:,:,None]), plt.title('seedling')
    plt.subplot(20, 7, 4 + (7*(current_image_index-1))), plt.imshow(resized_rgb_images_list[current_image_index] * (1 - seedling_mask[:,:,None])), plt.title('background')
    plt.subplot(20, 7, 5 + (7*(current_image_index-1))), plt.imshow(seedling_mask, cmap = 'jet', vmin=0, vmax=1), plt.title(f'Probability : mean = {seedling_mask.sum() / IMAGE_SHAPE.prod() :2f}'), plt.colorbar()
    plt.subplot(20, 7, 6 + (7*(current_image_index-1))), plt.imshow(seedling_color_confidence, cmap = 'jet', vmin=0, vmax=1), plt.title('Color Confidence'), plt.colorbar()
    plt.subplot(20, 7, 7 + (7*(current_image_index-1))), plt.hist(seedling_color_confidence.reshape(-1,1), bins=100, log=True, range=[0, 1]), plt.title('mean = ' + str(mean_seedling_color_confidence))
        
    # If the mean seedling color confidence is below the threshold, it indicates that there are not enough color values for initializing seedling color models.
    if mean_seedling_color_confidence < MEAN_SEEDLING_COLOR_CONFIDENCE_THRESHOLD:
        current_image_index += 1
    else:
        break
    
# Calculate initial seedling color model
init_mean_seedling_color, init_covariance_seedling_color = calculateMeanAndCovariance(current_image_color_vector, seedling_color_confidence)

# Fit a Gaussian Mixture Model to the background color values and initialize color model parameters
gmm = GaussianMixture(n_components = MAX_NUM_GAUSSIAN_MODELS - 1)
gmm.fit(ycbcr_vectors_list[0])
init_mean_background_color_models, init_covariance_background_color_models = gmm.means_, gmm.covariances_
init_color_model_means = np.vstack([init_mean_seedling_color, init_mean_background_color_models])
init_color_model_covariances = np.vstack([np.expand_dims(init_covariance_seedling_color, axis=0), init_covariance_background_color_models])
    
# Visualization
plot_ellipsoid([[init_mean_seedling_color, init_mean_background_color]], [[init_covariance_seedling_color, init_covariance_background_color]], ['Initial color model at '+ image_names_list[current_image_index]])


### Init single kernel

In [ ]:
# Create meshgrid of row and column indices based on tray dimensions
tray_row_indices, tray_col_indices = np.meshgrid(range(TRAY_ROW_COUNT), range(TRAY_COLUMN_COUNT))
# Stack row and column indices to form a 2D array representing cell indices in the tray
tray_cell_indices = np.stack([tray_row_indices.T, tray_col_indices.T], axis=-1)
# Calculate the centroid of each cell in the kernel
kernel_centroid = HALF_CELL_SHAPE + tray_cell_indices * CELL_SHAPE
# Calculate the top-left and bottom-right positions of the kernel for each cell
single_kernel_start = kernel_centroid - HALF_CELL_SHAPE
single_kernel_end = kernel_centroid + HALF_CELL_SHAPE + (CELL_SHAPE % 2)
# Adjust kernel positions to ensure they stay within image boundaries
single_kernel_start = np.where(single_kernel_start < 0, 0, single_kernel_start)
single_kernel_end = np.where(single_kernel_end < IMAGE_SHAPE, single_kernel_end, IMAGE_SHAPE)
# Create position vectors for each pixel in a cell (relative to the cell's center)
pixel_row_indices, pixel_col_indices = np.meshgrid(range(-HALF_CELL_HEIGHT, HALF_CELL_HEIGHT), range(-HALF_CELL_WIDTH, HALF_CELL_WIDTH))
pixel_positions_single_cell = np.stack([pixel_row_indices.T, pixel_col_indices.T], axis=-1)

### Update the seedling model from the day before the seedling color can be determined

In [ ]:
# Create prior probability for target positions in the spatial domain to find seedling color vectors
init_prior_probability_position_all_cells = createPrior(INIT_COVARIANCE_PRIOR_SCALING, IMAGE_SHAPE, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)
# Initialize prior probability for target positions in the spatial domain
prior_probability_position_all_cells = np.zeros(IMAGE_SHAPE)

plt.figure(figsize=(50, 50))

for init_image_index in range(1, current_image_index + 1):
    
    current_image = ycbcr_images_list[init_image_index]
    current_image_color_vector = ycbcr_vectors_list[init_image_index]
    
    # Calculate the likelihood of each pixel belonging to the foreground (seedling) color model
    foreground_likelihood_all_cells = calculateMultiGaussianDist(current_image, init_color_model_means[0], init_color_model_covariances[0])

    # Initialize background likelihood as zero and accumulate likelihoods from background color models
    number_of_color_models_buffer[init_image_index] = MAX_NUM_GAUSSIAN_MODELS
    background_likelihood_all_cells = np.zeros(IMAGE_SHAPE)
    for index_background_clusters in range (1, MAX_NUM_GAUSSIAN_MODELS):
        background_likelihood_all_cells += calculateMultiGaussianDist(current_image, init_color_model_means[index_background_clusters], init_color_model_covariances[index_background_clusters])

    # Calculate seedling probability for all cells using Bayesian segmentation
    seedling_probability_all_cells = calculateBayes(foreground_likelihood_all_cells * init_prior_probability_position_all_cells, background_likelihood_all_cells * (SPATIAL_BACKGROUND_PRIOR_SCALING * init_prior_probability_position_all_cells.max() - init_prior_probability_position_all_cells) / (MAX_NUM_GAUSSIAN_MODELS - 1))
    
    # If there are enough seedling pixels to calculate mean and covariance matrix of seedling color
    if seedling_probability_all_cells.sum() > MIN_SEEDLING_PIXEL_IN_IMAGE_FOR_CALCULATION:
                
        # Calculate the mean and covariance for the seedling color model
        # mean_seedling_color, cov_seedling_color = calculateMeanAndCovariance(current_image, seedling_probability_all_cells)    
        mean_seedling_color, cov_seedling_color = calculateMeanAndCovariance(current_image[np.where(seedling_probability_all_cells > PROBABILITY_THRESHOLD_FOR_UPDATE_COLOR_MODEL)])    
        
        # Fit a Gaussian mixture model to the background pixels
        gmm = GaussianMixture(n_components = MAX_NUM_GAUSSIAN_MODELS - 1)
        gmm.fit(current_image[np.where(seedling_probability_all_cells < PROBABILITY_THRESHOLD_FOR_UPDATE_COLOR_MODEL)])
        mean_background_color, cov_background_color = gmm.means_, gmm.covariances_

        # Store the updated color model means and covariances
        color_model_means_buffer[init_image_index] = np.vstack([mean_seedling_color, mean_background_color])
        color_model_covariances_buffer[init_image_index] = np.vstack([np.expand_dims(cov_seedling_color, axis=0), cov_background_color])
                 
        # Iterate over each cell in the tray
        for row_index in range(TRAY_ROW_COUNT):
            for col_index in range(TRAY_COLUMN_COUNT):

                # Extract the seedling probability and image data for the current cell's kernel region
                kernel_seedling_probability = seedling_probability_all_cells[single_kernel_start[row_index, col_index, 0]:single_kernel_end[row_index, col_index, 0],
                                                    single_kernel_start[row_index, col_index, 1]:single_kernel_end[row_index, col_index, 1]].copy()
                kernel_image = current_image[single_kernel_start[row_index, col_index, 0]:single_kernel_end[row_index, col_index, 0],
                                                                   single_kernel_start[row_index, col_index, 1]:single_kernel_end[row_index, col_index, 1]].copy()
                
                # Calculate and store the seedling size (sum of probabilities) and size-to-cell ratio
                seedling_size_buffer[init_image_index, row_index, col_index] = kernel_seedling_probability.sum()
                seedling_size_to_cell_ratio_buffer[init_image_index, row_index, col_index] = np.mean(kernel_seedling_probability)
                
                # If the size-to-cell ratio exceeds the threshold, the cell is considered to contain a seedling
                if seedling_size_to_cell_ratio_buffer[init_image_index, row_index, col_index] > SEEDLING_SIZE_TO_CELL_RATIO_THRESHOLD:
                    
                    seedling_status_buffer[init_image_index, row_index, col_index] = 1
                    
                    # Calculate the mean and covariance in the spatial domain for the seedling's position
                    seedling_position_model_means_buffer[init_image_index, row_index, col_index], seedling_position_model_covariances_buffer[init_image_index, row_index, col_index] = calculateMeanAndCovariance(pixel_positions_single_cell, kernel_seedling_probability)
                    
                    # Calculate and store the mean and covariance for the seedling's color
                    seedling_color_model_each_cell_means_buffer[init_image_index, row_index, col_index], seedling_color_model_each_cell_covariances_buffer[init_image_index, row_index, col_index] = calculateMeanAndCovariance(kernel_image, kernel_seedling_probability)

                    # Calculate the ellipse area based on the seedling's position covariance
                    ellipse_size_buffer[init_image_index, row_index, col_index] = calculateEllipseArea(seedling_position_model_covariances_buffer[init_image_index, row_index, col_index])

                # Update the prior probability when image index is current image index based on the current cell's seedling status
                if init_image_index == current_image_index:
                    if seedling_status_buffer[init_image_index, row_index, col_index] == 1:
                        prior_probability_position_all_cells[single_kernel_start[row_index, col_index, 0]:single_kernel_end[row_index, col_index, 0],
                                                            single_kernel_start[row_index, col_index, 1]:single_kernel_end[row_index, col_index, 1]] = calculateMultiGaussianDist(pixel_positions_single_cell, seedling_position_model_means_buffer[init_image_index, row_index, col_index], seedling_position_model_covariances_buffer[init_image_index, row_index, col_index] * COVARIANCE_PRIOR_ALL_SEEDLING_CELL_SCALING)
                    else:
                        prior_probability_position_all_cells[single_kernel_start[row_index, col_index, 0]:single_kernel_end[row_index, col_index, 0],
                                                            single_kernel_start[row_index, col_index, 1]:single_kernel_end[row_index, col_index, 1]] = calculateMultiGaussianDist(pixel_positions_single_cell, np.zeros(2), np.array([[CELL_HEIGHT, 0], [0, CELL_WIDTH]]) * COVARIANCE_PRIOR_EMPTY_CELL_SCALING)
        
        if np.any(seedling_status_buffer[init_image_index]):
            
            # Calculate Euclidean distance between the seedling's center and the planting cell's center for each cell in each image
            # distance_position = np.sqrt(np.square(seedling_position_model_means_buffer[init_image_index]).sum(axis = -1))
            distance_position = calculateEuclidean(seedling_position_model_means_buffer[init_image_index], np.zeros_like(seedling_position_model_means_buffer[init_image_index]))
            distance_position_buffer[init_image_index] = np.where(seedling_status_buffer[init_image_index] == 0, 0, distance_position)
            # Calculate Euclidean distance between the mean Cb-Cr values of each seedling and the mean Cb-Cr of the entire tray in YCbCr color space for each cell in each image
            # distance_color= np.sqrt(np.square(seedling_color_model_each_cell_means_buffer[init_image_index, :, :, 1:] - color_model_means_buffer[init_image_index, 0, 1:]).sum(axis = -1))
            distance_color = calculateEuclidean(seedling_color_model_each_cell_means_buffer[init_image_index, :, :, 1:], color_model_means_buffer[init_image_index, 0, 1:])
            distance_color_buffer[init_image_index] = np.where(seedling_status_buffer[init_image_index] == 0, 0, distance_color)
            
            # Update confidence for seedling status based on previous confidence, as well as color and position distances
            seedling_status_confidence = np.where(((seedling_status_confidence_buffer[init_image_index - 1] < INIT_CONFIDENCE).astype(int) * (seedling_status_buffer[init_image_index]).astype(int)), INIT_CONFIDENCE, seedling_status_confidence_buffer[init_image_index - 1])
            
            seedling_status = np.where(seedling_status_buffer[init_image_index] == 1, STATUS_UPDATE_STATUS_CONFIDENCE_WEIGHT, 1 - STATUS_UPDATE_STATUS_CONFIDENCE_WEIGHT)
            
            # max_seedling_color_distance is the minimum Euclidean distance between the mean color of the seedling and the nearest background color model in YCbCr space
            # max_seedling_color_distance = (np.sqrt(np.square(color_model_means_buffer[init_image_index, 1:number_of_color_models_buffer[init_image_index]] - color_model_means_buffer[init_image_index, 0]).sum(axis=-1))).min()
            max_seedling_color_distance = calculateEuclidean(color_model_means_buffer[init_image_index, 1:number_of_color_models_buffer[init_image_index]], color_model_means_buffer[init_image_index, 0]).min()
            # Calculate color_distance_scaling using a linear equation: 
            # If the color distance is 0, scaling equals COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT. If the color distance equals max_seedling_color_distance, scaling equals 1 - COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT.
            color_distance_scaling = COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT - ( ( 2 * COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT - 1) * distance_color_buffer[init_image_index] / max_seedling_color_distance )  
            color_distance_scaling = np.clip(np.where(seedling_status_buffer[init_image_index] == 0, 0.5, color_distance_scaling), a_max= COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT, a_min= 1-COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT)

            # max_seedling_position_distance is the Euclidean distance from the center of the cell to the corner of the cell
            # max_seedling_position_distance = np.sqrt(np.square(HALF_CELL_SHAPE).sum())
            max_seedling_position_distance = calculateEuclidean(HALF_CELL_SHAPE, np.zeros_like(HALF_CELL_SHAPE))
            # Calculate position_distance_scaling using a linear equation:
            # If the position distance is 0, scaling equals POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT. If the position distance equals max_seedling_position_distance, scaling equals 1 - POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT.
            position_distance_scaling = POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT - ( ( 2 * POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT - 1) * distance_position_buffer[init_image_index] / max_seedling_position_distance )
            position_distance_scaling = np.clip(np.where(seedling_status_buffer[init_image_index] == 0, 0.5, position_distance_scaling), a_max= POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT, a_min= 1-POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT)
            
            # Update confidence
            foreground_probability = seedling_status_confidence * seedling_status * color_distance_scaling * position_distance_scaling
            background_probability = (1 - seedling_status_confidence * BACKGROUND_CONFIDENCE_WEIGHT) * (1 - seedling_status) * (1 - color_distance_scaling) * (1 - position_distance_scaling)
            seedling_status_confidence_buffer[init_image_index] = calculateBayes( foreground_probability, background_probability )
            
            # Visualization
            ellipse_image = displayEllipseOnImage(resized_rgb_images_list[init_image_index], seedling_status_buffer[init_image_index], seedling_position_model_means_buffer[init_image_index], seedling_position_model_covariances_buffer[init_image_index], TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)
            plt.subplot(current_image_index,7,((init_image_index-1)*7)+5), plt.imshow(ellipse_image), plt.title(f'num_gaussian={number_of_color_models_buffer[init_image_index]}')
            plt.subplot(current_image_index,7,((init_image_index-1)*7)+6), plt.imshow(seedling_status_confidence_buffer[init_image_index], vmax=1, vmin=0, cmap='jet'), plt.colorbar(), plt.title('seedling status confidence')
            plt.subplot(current_image_index,7,((init_image_index-1)*7)+7), plt.hist(seedling_status_confidence_buffer[init_image_index].reshape(-1,1), range=[0,1], log=True, bins=40), plt.title('seedling status confidence')
            
        # Visualization
        plt.subplot(current_image_index,7,((init_image_index-1)*7)+2), plt.imshow(seedling_probability_all_cells, cmap='hot'), plt.title(f'Probability Image (sum={seedling_probability_all_cells.mean():2f})'),
        plt.subplot(current_image_index,7,((init_image_index-1)*7)+1), plt.imshow(rgb_images_list[init_image_index], cmap='hot'),plt.title(f'{init_image_index} : {image_names_list[init_image_index]}')
        plt.subplot(current_image_index,7,((init_image_index-1)*7)+3), plt.imshow(resized_rgb_images_list[init_image_index] * seedling_probability_all_cells[:,:,None]), plt.title('Seedling')
        plt.subplot(current_image_index,7,((init_image_index-1)*7)+4), plt.imshow(resized_rgb_images_list[init_image_index] * (1-seedling_probability_all_cells[:,:,None])), plt.title('Background')

### Init expanded kernel

In [ ]:
# init kernel for calculate seedling probability of each cell
# Create meshgrid of row and column indices based on tray dimensions
tray_row_indices, tray_col_indices = np.meshgrid(range(TRAY_ROW_COUNT), range(TRAY_COLUMN_COUNT))
# Stack row and column indices to form a 2D array representing cell indices in the tray
tray_cell_indices = np.stack([tray_row_indices.T, tray_col_indices.T], axis=-1)
# Calculate the centroid of each cell in the kernel
kernel_centroid = HALF_CELL_SHAPE + tray_cell_indices * CELL_SHAPE
# Calculate the top-left and bottom-right positions of the kernel for each cell
expanded_kernel_start = kernel_centroid - (HALF_CELL_SHAPE * 3)
expanded_kernel_end = kernel_centroid + (HALF_CELL_SHAPE * 3) + (CELL_SHAPE % 2)
# Adjust kernel positions to ensure they stay within image boundaries
expanded_kernel_start = np.where(expanded_kernel_start < 0, 0, expanded_kernel_start)
expanded_kernel_end = np.where(expanded_kernel_end < IMAGE_SHAPE, expanded_kernel_end, IMAGE_SHAPE)

### Update seedling model

In [ ]:
# Backup data for rerunning the program
current_image_index_l = current_image_index
prior_probability_position_all_cells_l = prior_probability_position_all_cells.copy()

In [ ]:
# Restore saved data for rerunning the process with previously backed-up state
current_image_index  = current_image_index_l
prior_probability_position_all_cells = prior_probability_position_all_cells_l.copy()

# Visualization
plt.figure(figsize=(35,120))

# Begin iterating through the remaining images
while(current_image_index < number_of_images - 1): 
     
    current_image_index += 1
    
    # Retrieve current image and associated color data
    current_image = ycbcr_images_list[current_image_index]
    current_image_color_vector = ycbcr_vectors_list[current_image_index]
    # Retrieve Gaussian mixture model parameters from the buffer
    color_model_means_previous = color_model_means_buffer[current_image_index - 1,:number_of_color_models_buffer[current_image_index - 1]]
    color_model_covariances_previous = color_model_covariances_buffer[current_image_index - 1,:number_of_color_models_buffer[current_image_index - 1]]
    
    # Fit a Gaussian Mixture Model (GMM) to the background pixels of the current image
    gmm = GaussianMixture(n_components = number_of_color_models_buffer[current_image_index - 1], means_init = color_model_means_previous)
    gmm.fit(current_image_color_vector)
    gmm_means = gmm.means_
    gmm_covariances = gmm.covariances_
    
    # Align GMM models with existing color models, ensuring the seedling model is first.
    pair_gmm_indices = pairGaussianModels(color_model_means_previous, color_model_covariances_previous, gmm_means, gmm_covariances)
    gmm_means = gmm_means[pair_gmm_indices[1]]
    gmm_covariances = gmm_covariances[pair_gmm_indices[1]]

    # Compute likelihoods for the foreground (seedling) and background using color models and GMMs
    pdf_seedling_color_model = calculateMultiGaussianDist(current_image, color_model_means_previous[0], color_model_covariances_previous[0])
    pdf_seedling_gmm = calculateMultiGaussianDist(current_image, gmm_means[0], gmm_covariances[0])
    foreground_likelihood_all_cells = pdf_seedling_color_model * pdf_seedling_gmm
    
    background_likelihood_all_cells = np.zeros(IMAGE_SHAPE)
    for index_background_clusters in range (1, number_of_color_models_buffer[current_image_index - 1]):
        pdf_background_color_model = calculateMultiGaussianDist(current_image, color_model_means_previous[index_background_clusters], color_model_covariances_previous[index_background_clusters])
        pdf_background_gmm = calculateMultiGaussianDist(current_image, gmm_means[index_background_clusters], gmm_covariances[index_background_clusters])
        background_likelihood_all_cells += pdf_background_color_model * pdf_background_gmm 

    # Calculate the seedling probability for each cell using Bayes' theorem
    seedling_probability_all_cells = calculateBayes(foreground_likelihood_all_cells * prior_probability_position_all_cells, background_likelihood_all_cells * (SPATIAL_BACKGROUND_PRIOR_SCALING * prior_probability_position_all_cells.max() - prior_probability_position_all_cells) / (number_of_color_models_buffer[current_image_index - 1] - 1))
    
    
    # Compute the mean and covariance for seedling color pixels
    # mean_seedling_color, cov_seedling_color = calculateMeanAndCovariance(current_image, seedling_probability_all_cells)    
    mean_seedling_color, cov_seedling_color = calculateMeanAndCovariance(current_image[np.where(seedling_probability_all_cells > PROBABILITY_THRESHOLD_FOR_UPDATE_COLOR_MODEL)])    

    # Initialize an array to store the minimum Euclidean and Bhattacharyya distances for each number of Gaussian models
    # seedling_color_gaussian_model_distance = np.zeros((2, number_of_color_models_buffer[current_image_index - 1] - MIN_NUM_GAUSSIAN_MODELS + 1))
    seedling_color_gaussian_model_distance = np.zeros((3, number_of_color_models_buffer[current_image_index - 1] - MIN_NUM_GAUSSIAN_MODELS + 1))

    # Iterate over the range of possible numbers of Gaussian models
    for n_gaussians in range(MIN_NUM_GAUSSIAN_MODELS, number_of_color_models_buffer[current_image_index - 1] + 1):
        
        # Initialize and fit the Gaussian Mixture Model with the current number of components (Gaussian models)
        gmm = GaussianMixture(n_components=n_gaussians)
        gmm.fit(current_image_color_vector)
        
        labeled_vector = gmm.predict(current_image_color_vector)
        
        # Initialize arrays to store both Euclidean and Bhattacharyya distances for each cluster (Gaussian component)
        euclidean_distance_buffer = np.zeros(n_gaussians)
        bhattacharyya_distance_buffer = np.zeros(n_gaussians)
        mse_buffer = np.zeros(n_gaussians)
        
        # Calculate the Euclidean and Bhattacharyya distances between each cluster's color model and the seedling color model
        for cluster_index in range(n_gaussians):

            cluster_mask = np.where(labeled_vector == cluster_index, 1, 0).reshape(IMAGE_SHAPE)
            mse_buffer[cluster_index] = np.square(seedling_probability_all_cells - cluster_mask).mean()
            # mse_buffer[cluster_index] = np.where(seedling_probability_all_cells - cluster_mask > 0, 1, 0).sum()

            # Compute the Euclidean distance between the seedling and the current cluster
            # euclidean_distance_buffer[cluster_index] = np.sqrt(np.square(mean_seedling_color - gmm.means_[cluster_index]).sum())
            # euclidean_distance_buffer[cluster_index] = calculateEuclidean(color_model_means_buffer[current_image_index - 1, 0], gmm.means_[cluster_index]) 
            euclidean_distance_buffer[cluster_index] = calculateEuclidean(mean_seedling_color, gmm.means_[cluster_index]) 
            # Compute the Bhattacharyya distance between the seedling and the current cluster
            # bhattacharyya_distance_buffer[cluster_index] = calculateBhattacharyya(color_model_means_buffer[current_image_index - 1, 0], color_model_covariances_buffer[current_image_index - 2, 0], gmm.means_[cluster_index], gmm.covariances_[cluster_index])
            bhattacharyya_distance_buffer[cluster_index] = calculateBhattacharyya(mean_seedling_color, cov_seedling_color, gmm.means_[cluster_index], gmm.covariances_[cluster_index])

        # Store the minimum Euclidean and Bhattacharyya distances for the current number of Gaussian models
        seedling_color_gaussian_model_distance[0, n_gaussians - MIN_NUM_GAUSSIAN_MODELS] = euclidean_distance_buffer.min()
        seedling_color_gaussian_model_distance[1, n_gaussians - MIN_NUM_GAUSSIAN_MODELS] = bhattacharyya_distance_buffer.min()
        seedling_color_gaussian_model_distance[2, n_gaussians - MIN_NUM_GAUSSIAN_MODELS] = mse_buffer.min()
        
    # Determine the optimal number of Gaussian models based on the minimum value between both Euclidean and Bhattacharyya distances
    # number_of_color_models_buffer[current_image_index] = MIN_NUM_GAUSSIAN_MODELS + seedling_color_gaussian_model_distance.argmin(axis=1).min()
    number_of_color_models_buffer[current_image_index] = MIN_NUM_GAUSSIAN_MODELS + seedling_color_gaussian_model_distance[:2].argmin(axis=1).min()


    # Fit a new GMM with the selected number of components (excluding the seedling component) to update the background color models
    gmm = GaussianMixture(n_components = number_of_color_models_buffer[current_image_index] - 1)
    gmm.fit(current_image[np.where(seedling_probability_all_cells < PROBABILITY_THRESHOLD_FOR_UPDATE_COLOR_MODEL)])
    mean_background_color, cov_background_color = gmm.means_, gmm.covariances_
    
    # Update the color model with the seedling and background GMM parameters for the current image
    color_model_means_buffer[current_image_index, :number_of_color_models_buffer[current_image_index]] = np.vstack([mean_seedling_color, mean_background_color])
    color_model_covariances_buffer[current_image_index, :number_of_color_models_buffer[current_image_index]] = np.vstack([np.expand_dims(cov_seedling_color, axis=0), cov_background_color])
    
    prior_probability_position_all_cells = np.zeros(IMAGE_SHAPE)
    
    # Iterate over each cell in the tray
    for row_index in range(TRAY_ROW_COUNT):
        for col_index in range(TRAY_COLUMN_COUNT):
            
            # Calculate the boundaries of each pixel relative to the cell's center
            relative_top_left = expanded_kernel_start[row_index, col_index] - kernel_centroid[row_index, col_index]
            relative_bottom_right  = expanded_kernel_end[row_index, col_index] - kernel_centroid[row_index, col_index] 
            # Generate position vectors for each pixel within the cell
            pixel_row_indices, pixel_col_indices = np.meshgrid(range(relative_top_left[0], relative_bottom_right[0]), range(relative_top_left[1], relative_bottom_right[1]))
            pixel_positions_in_cell = np.stack([pixel_row_indices.T, pixel_col_indices.T], axis=-1)
            
            # Create prior probability in spatial domain based on the seedling status from the previous image
            if seedling_status_buffer[current_image_index - 1, row_index, col_index] == 1:

                # Scale the covariance matrix based on the size of the seedling ellipse
                covariance_scaling = MAX_COVARIANCE_PRIOR_EACH_SEEDLING_CELL_SCALING - COVARIANCE_PRIOR_EACH_SEEDLING_CELL_SCALING_RATIO * 1e-08 * ellipse_size_buffer[current_image_index - 1, row_index, col_index] ** 2
                covariance_scaling = np.where(covariance_scaling < MIN_COVARIANCE_PRIOR_EACH_SEEDLING_CELL_SCALING, MIN_COVARIANCE_PRIOR_EACH_SEEDLING_CELL_SCALING, covariance_scaling)
                # Calculate the prior probability of pixel positions in the cell based on a Gaussian distribution
                prior_probability_position_in_cell = calculateMultiGaussianDist(pixel_positions_in_cell, seedling_position_model_means_buffer[current_image_index - 1, row_index, col_index], seedling_position_model_covariances_buffer[current_image_index - 1, row_index, col_index] * covariance_scaling)
                
                # Get foreground and background likelihoods based on seedling probability in the current cell
                foreground_likelihood_in_cell = seedling_probability_all_cells[expanded_kernel_start[row_index, col_index, 0]:expanded_kernel_end[row_index, col_index, 0], expanded_kernel_start[row_index, col_index, 1]:expanded_kernel_end[row_index, col_index, 1]]
                background_likelihood_in_cell = 1 - foreground_likelihood_in_cell
                
                kernel_image = current_image[expanded_kernel_start[row_index, col_index, 0]:expanded_kernel_end[row_index, col_index, 0], expanded_kernel_start[row_index, col_index, 1]:expanded_kernel_end[row_index, col_index, 1]]
                
            
            elif seedling_status_buffer[current_image_index - 1, row_index, col_index] == 0:
                
                # Prior probability for an empty cell is based on a default Gaussian distribution
                prior_probability_position_in_cell = calculateMultiGaussianDist(pixel_positions_single_cell, np.zeros(2), np.array([[CELL_HEIGHT, 0], [0, CELL_WIDTH]]) * COVARIANCE_PRIOR_EMPTY_CELL_SCALING)
               
                # Get foreground and background likelihoods for an empty cell
                foreground_likelihood_in_cell = seedling_probability_all_cells[single_kernel_start[row_index, col_index, 0]:single_kernel_end[row_index, col_index, 0], single_kernel_start[row_index, col_index, 1]:single_kernel_end[row_index, col_index, 1]]
                background_likelihood_in_cell = 1 - foreground_likelihood_in_cell
                
                kernel_image = current_image[single_kernel_start[row_index, col_index, 0]:single_kernel_end[row_index, col_index, 0], single_kernel_start[row_index, col_index, 1]:single_kernel_end[row_index, col_index, 1]]
            
            # Calculate the seedling probability in the cell using Bayesian segmentation
            seedling_probability_each_cell = calculateBayes(foreground_likelihood_in_cell * prior_probability_position_in_cell , background_likelihood_in_cell * (SPATIAL_BACKGROUND_PRIOR_SCALING * prior_probability_position_in_cell.max() - prior_probability_position_in_cell) / (number_of_color_models_buffer[current_image_index] - 1))
            
            # Calculate seedling size as a ratio to the cell size and update the buffer
            seedling_size_to_cell_ratio_buffer[current_image_index, row_index, col_index] = seedling_probability_each_cell.sum() / CELL_SHAPE.prod()
            seedling_size_buffer[current_image_index, row_index, col_index] = seedling_probability_each_cell.sum()


            # Check if seedling size exceeds the threshold
            if seedling_size_to_cell_ratio_buffer[current_image_index, row_index, col_index] > SEEDLING_SIZE_TO_CELL_RATIO_THRESHOLD:

                # Update position means and covariances based on seedling status
                if seedling_status_buffer[current_image_index - 1, row_index, col_index] == 1: 
                    position_means, position_covariances = calculateMeanAndCovariance(pixel_positions_in_cell, seedling_probability_each_cell)
                elif seedling_status_buffer[current_image_index - 1, row_index, col_index] == 0: 
                    position_means, position_covariances = calculateMeanAndCovariance(pixel_positions_single_cell, seedling_probability_each_cell)
                
                # Ensure the seedling's center remains within its cell
                if np.all((position_means > pixel_positions_single_cell[0,0]) * (position_means < pixel_positions_single_cell[-1,-1])):
                    seedling_status_buffer[current_image_index, row_index, col_index] = 1
                else:
                    print(f'{image_names_list[current_image_index], row_index, col_index} center out of cell = {position_means}')
                    seedling_status_buffer[current_image_index, row_index, col_index] = 0

            else: 
                # If seedling size is below threshold, flag the cell as not having started germination
                seedling_status_buffer[current_image_index, row_index, col_index] = 0


            # If seedling is present in the current image, update the position and covariance buffers
            if seedling_status_buffer[current_image_index, row_index, col_index] == 1:
                
                seedling_position_model_means_buffer[current_image_index, row_index, col_index], seedling_position_model_covariances_buffer[current_image_index, row_index, col_index] = position_means.copy(), position_covariances.copy()
                # Update the prior probability in the spatial domain based on the current Gaussian distribution
                prior_probability_position_all_cells[expanded_kernel_start[row_index, col_index, 0]:expanded_kernel_end[row_index, col_index, 0],
                                                expanded_kernel_start[row_index, col_index, 1]:expanded_kernel_end[row_index, col_index, 1]] += calculateMultiGaussianDist(pixel_positions_in_cell, seedling_position_model_means_buffer[current_image_index, row_index, col_index], seedling_position_model_covariances_buffer[current_image_index, row_index, col_index] * COVARIANCE_PRIOR_ALL_SEEDLING_CELL_SCALING)
                # Update the ellipse size buffer and color model for the cell
                ellipse_size_buffer[current_image_index, row_index, col_index] = calculateEllipseArea(seedling_position_model_covariances_buffer[current_image_index, row_index, col_index])
                seedling_color_model_each_cell_means_buffer[current_image_index, row_index, col_index], seedling_color_model_each_cell_covariances_buffer[current_image_index, row_index, col_index] = calculateMeanAndCovariance(kernel_image, seedling_probability_each_cell)
                
            else:
                # If no seedling is present, update the prior probability using an empty cell model
                prior_probability_position_all_cells[expanded_kernel_start[row_index, col_index, 0]:expanded_kernel_end[row_index, col_index, 0],
                                expanded_kernel_start[row_index, col_index, 1]:expanded_kernel_end[row_index, col_index, 1]] += calculateMultiGaussianDist(pixel_positions_in_cell, np.zeros(2), np.array([[CELL_HEIGHT, 0], [0, CELL_WIDTH]]) * COVARIANCE_PRIOR_EMPTY_CELL_SCALING)

    # Calculate Euclidean distance between the seedling's center and the planting cell's center for each cell in each image
    # distance_position = np.sqrt(np.square(seedling_position_model_means_buffer[current_image_index]).sum(axis = -1))
    distance_position = calculateEuclidean(seedling_position_model_means_buffer[current_image_index], np.zeros_like(seedling_position_model_means_buffer[current_image_index]))
    distance_position_buffer[current_image_index] = np.where(seedling_status_buffer[current_image_index] == 0, 0, distance_position)
    # Calculate Euclidean distance between the mean Cb-Cr values of each seedling and the mean Cb-Cr of the entire tray in YCbCr color space for each cell in each image
    # distance_color= np.sqrt(np.square(seedling_color_model_each_cell_means_buffer[current_image_index, :, :, 1:] - color_model_means_buffer[current_image_index, 0, 1:]).sum(axis = -1))
    distance_color = calculateEuclidean(seedling_color_model_each_cell_means_buffer[current_image_index, :, :, 1:], color_model_means_buffer[current_image_index, 0, 1:])
    distance_color_buffer[current_image_index] = np.where(seedling_status_buffer[current_image_index] == 0, 0, distance_color)
    
    # Update confidence for seedling status based on previous confidence, as well as color and position distances
    seedling_status_confidence = np.where(((seedling_status_confidence_buffer[current_image_index - 1] < INIT_CONFIDENCE).astype(int) * (seedling_status_buffer[current_image_index]).astype(int)), INIT_CONFIDENCE, seedling_status_confidence_buffer[current_image_index - 1])
    seedling_status = np.where(seedling_status_buffer[current_image_index] == 1, STATUS_UPDATE_STATUS_CONFIDENCE_WEIGHT, 1 - STATUS_UPDATE_STATUS_CONFIDENCE_WEIGHT)
    
    # max_seedling_color_distance is the minimum Euclidean distance between the mean color of the seedling and the nearest background color model in YCbCr space
    max_seedling_color_distance = calculateEuclidean(color_model_means_buffer[current_image_index, 1:number_of_color_models_buffer[current_image_index]], color_model_means_buffer[current_image_index, 0]).min()
    # Calculate color_distance_scaling using a linear equation: 
    # If the color distance is 0, scaling equals COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT. If the color distance equals max_seedling_color_distance, scaling equals 1 - COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT.
    color_distance_scaling = COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT - ( ( 2 * COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT - 1) * distance_color_buffer[current_image_index] / max_seedling_color_distance )  
    color_distance_scaling = np.clip(np.where(seedling_status_buffer[current_image_index] == 0, 0.5, color_distance_scaling), a_max= COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT, a_min= 1-COLOR_UPDATE_STATUS_CONFIDENCE_WEIGHT)
    
    # max_seedling_position_distance is the Euclidean distance from the center of the cell to the corner of the cell
    # max_seedling_position_distance = np.sqrt(np.square(HALF_CELL_SHAPE).sum())
    max_seedling_position_distance = calculateEuclidean(HALF_CELL_SHAPE, np.zeros_like(HALF_CELL_SHAPE))
    # Calculate position_distance_scaling using a linear equation:
    # If the position distance is 0, scaling equals POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT. If the position distance equals max_seedling_position_distance, scaling equals 1 - POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT.
    position_distance_scaling = POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT - ( ( 2 * POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT - 1) * distance_position_buffer[current_image_index] / max_seedling_position_distance )
    position_distance_scaling = np.clip(np.where(seedling_status_buffer[current_image_index] == 0, 0.5, position_distance_scaling), a_max= POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT, a_min= 1-POSITION_UPDATE_STATUS_CONFIDENCE_WEIGHT)
    
    # Update confidence
    foreground_probability = seedling_status_confidence * seedling_status * color_distance_scaling * position_distance_scaling
    background_probability = (1 - seedling_status_confidence * BACKGROUND_CONFIDENCE_WEIGHT) * (1 - seedling_status) * (1 - color_distance_scaling) * (1 - position_distance_scaling)
    seedling_status_confidence_buffer[current_image_index] = calculateBayes( foreground_probability, background_probability )

    # Visualization
    ellipse_image = displayEllipseOnImage(resized_rgb_images_list[current_image_index].copy(), seedling_status_buffer[current_image_index], seedling_position_model_means_buffer[current_image_index], seedling_position_model_covariances_buffer[current_image_index], TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)
    # title_num_gaussian = f'num_gaussian={number_of_color_models_buffer[current_image_index]}' if number_of_color_models_buffer[current_image_index] == number_of_color_models_buffer[current_image_index-1] else f'num_gaussian={number_of_color_models_buffer[current_image_index]} (e,b = {MIN_NUM_GAUSSIAN_MODELS+seedling_color_gaussian_model_distance.argmin(axis=1)})'
    title_num_gaussian = f'num_gaussian={number_of_color_models_buffer[current_image_index]} (e,b,m = {MIN_NUM_GAUSSIAN_MODELS+seedling_color_gaussian_model_distance.argmin(axis=1)})'
    plt.subplot(35,5,5*(current_image_index - current_image_index_l)-2), plt.imshow(ellipse_image), plt.title(title_num_gaussian)
    plt.subplot(35,5,5*(current_image_index - current_image_index_l)-1), plt.imshow(seedling_status_confidence_buffer[current_image_index], vmax=1, vmin=0, cmap='jet'), plt.colorbar(), plt.title('seedling status confidence')
    plt.subplot(35,5,5*(current_image_index - current_image_index_l)),   plt.hist(seedling_status_confidence_buffer[current_image_index].reshape(-1,1), range=[0,1], log=True, bins=40), plt.title('seedling status confidence')
    plt.subplot(35,5,5*(current_image_index - current_image_index_l)-3), plt.imshow(seedling_probability_all_cells, 'hot'), plt.title(f'Prob : mean={seedling_probability_all_cells.mean()}')
    plt.subplot(35,5,5*(current_image_index - current_image_index_l)-4), plt.imshow(resized_rgb_images_list[current_image_index]), plt.title(f'{current_image_index} : {image_names_list[current_image_index]}')
    

In [ ]:
plot_ellipsoid_color_model(color_model_means_buffer,  color_model_covariances_buffer, image_names_list)

## Report

In [ ]:
plt.figure(figsize=(20,20))
for index in range(number_of_images):
    ellipse_image = displayEllipseOnImage(resized_rgb_images_list[index].copy(), seedling_status_buffer[index], seedling_position_model_means_buffer[index], seedling_position_model_covariances_buffer[index], TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)
    plt.subplot(7,5,1+index), plt.imshow(ellipse_image), plt.title(image_names_list[index].split(".")[0])

In [ ]:
n = 4
plt.figure(figsize=(30,6*n))
plt.subplot(n,1,1), plt.plot(image_names_list, seedling_status_confidence_buffer.reshape(number_of_images, -1).mean(axis=-1)), plt.title('mean seedling_status_confidence'), plt.ylim(0, 1), plt.grid(linewidth = "0.1")
plt.subplot(n,1,2), plt.plot(image_names_list, seedling_status_buffer.mean(axis=-1).mean(axis=-1)), plt.title('mean seedling_status'), plt.ylim(0, 1), plt.grid(linewidth = "0.1")
plt.subplot(n,1,3)
plt.plot(image_names_list, ellipse_size_buffer.reshape(number_of_images, -1).mean(axis = -1), label = 'mean ellipse_size')
plt.plot(image_names_list, seedling_size_buffer.reshape(number_of_images, -1).mean(axis = -1), label = 'mean seedling_size')
plt.legend(), plt.title('mean size'), plt.grid(linewidth = "0.1")
plt.subplot(n,3,4*3-2), plt.plot(image_names_list, color_model_means_buffer[:,0,0]), plt.title('Y'), plt.ylim(0, 1), plt.grid(linewidth = "0.1")
plt.subplot(n,3,4*3-1), plt.plot(image_names_list, color_model_means_buffer[:,0,1]), plt.title('Cb'), plt.ylim(-0.5, 0.5), plt.grid(linewidth = "0.1")
plt.subplot(n,3,4*3), plt.plot(image_names_list, color_model_means_buffer[:,0,2]), plt.title('Cr'), plt.ylim(-0.5, 0.5), plt.grid(linewidth = "0.1")

In [ ]:
colorArray = (convertYCbCrtoRGB(color_model_means_buffer[:,0,:]))
# print( colorArray.shape[ 0 ] )
colorArrayImage = np.zeros( ( colorArray.shape[0], 10, colorArray.shape[1] )  )
colorArrayImage.shape
for replaceTimes in range(colorArrayImage.shape[ 1 ] ):
    colorArrayImage[ :, replaceTimes, : ] = colorArray[ :, : ]
plt.imshow( colorArrayImage )
# print( colorArrayImage.shape( ))

In [ ]:
plt.imshow(np.rot90(np.tile(colorArray.reshape(34,1,3), (1,5,1))))

In [ ]:
plt.figure(figsize=(30,8))
plt.subplot(2,5,1), plt.imshow(resized_rgb_images_list[current_image_index]),  plt.title(f'index = {current_image_index}, {image_names_list[current_image_index]}')
plt.subplot(2,5,6), plt.imshow(ellipse_image),  plt.title(f'ellipse')
plt.subplot(2,5,2), plt.imshow(seedling_status_confidence_buffer[current_image_index], cmap='hot'), plt.colorbar(), plt.title(f'seedling status confidence')
plt.subplot(2,5,7), plt.hist(seedling_status_confidence_buffer[current_image_index].reshape(-1,1), range=[0,1], log=True, bins=40)
plt.subplot(2,5,3), plt.imshow(seedling_size_buffer[current_image_index], cmap='hot'), plt.colorbar(), plt.title(f'seedling size')
plt.subplot(2,5,8), plt.hist(seedling_size_buffer[current_image_index].reshape(-1,1), log=True, bins=40)
plt.subplot(2,5,4), plt.imshow(distance_color_buffer[current_image_index], cmap='hot'), plt.colorbar(), plt.title(f'euclidean distance between mean cb-cr of each seedling \nand mean cb-cr of entire tray in ycbcr color space')
plt.subplot(2,5,9), plt.hist(distance_color_buffer[current_image_index].reshape(-1,1), log=True, bins=40)
plt.subplot(2,5,5), plt.imshow(distance_position_buffer[current_image_index], cmap='hot'), plt.colorbar(), plt.title(f'euclidean distance between seedling center and planting cell center')
plt.subplot(2,5,10), plt.hist(distance_position_buffer[current_image_index].reshape(-1,1), log=True, bins=40)

In [ ]:
status_confidence_over_threshold = seedling_status_confidence_buffer[current_image_index] > SEEDLING_STATUS_CONFIDENCE_THRESHOLD

# Cells with seedling probability over the threshold
over_threshold_count = np.count_nonzero(status_confidence_over_threshold)
over_threshold_percentage = over_threshold_count / (TRAY_COLUMN_COUNT * TRAY_ROW_COUNT)
over_threshold_cells = np.stack(np.where(status_confidence_over_threshold), axis=-1)

# Cells with seedling probability under the threshold
under_threshold_count = np.count_nonzero(~status_confidence_over_threshold)
under_threshold_percentage = under_threshold_count / (TRAY_COLUMN_COUNT * TRAY_ROW_COUNT)
under_threshold_cells = np.stack(np.where(~status_confidence_over_threshold), axis=-1)

# Seedling size analysis (Outliers)
size_outliers = checkOutlier(status_confidence_over_threshold, seedling_size_buffer[current_image_index])
mean_size = seedling_size_buffer[current_image_index][status_confidence_over_threshold].mean() * PIXELS_TO_REAL_DISTANCE_RATIO

# Seedling color analysis (Outliers)
color_outliers = checkOutlier(status_confidence_over_threshold, distance_color_buffer[current_image_index])
mean_color_rgb = convertYCbCrtoRGB(color_model_means_buffer[current_image_index][0])

# Generate report-like output
print(f"Seedling Probability")
print(f"Cells with Seedling Probability > {SEEDLING_STATUS_CONFIDENCE_THRESHOLD}:")
print(f"  Total: {over_threshold_count} cells ({over_threshold_percentage:.2%})")
print(f"  Locations: {', '.join(str(k) for k in over_threshold_cells)}")

print(f"\nCells with Seedling Probability ≤ {SEEDLING_STATUS_CONFIDENCE_THRESHOLD}:")
print(f"  Total: {under_threshold_count} cells ({under_threshold_percentage:.2%})")
print(f"  Locations: {', '.join(str(k) for k in under_threshold_cells)}")

print("\nSeedling Size ")
print(f"  Mean Seedling Size: {mean_size:.2f} mm²")
print(f"  Size outliers detected at cells: {', '.join(str(k) for k in size_outliers)}")

print("\nSeedling Color ")
print(f"  Mean Seedling Color (RGB): {mean_color_rgb}")
print(f"  Color outliers detected at cells: {', '.join(str(k) for k in color_outliers)}")

plt.figure(figsize=(15,8))
plt.subplot(221), plt.imshow(resized_rgb_images_list[current_image_index]),  plt.title(f'{image_names_list[current_image_index]}')
plt.subplot(222), plt.imshow(status_confidence_over_threshold, 'gray'),  plt.title(f'Cells with seedling probability over {SEEDLING_STATUS_CONFIDENCE_THRESHOLD}')
error_image = np.zeros((TRAY_ROW_COUNT, TRAY_COLUMN_COUNT))
error_image[size_outliers[:, 0], size_outliers[:, 1]] = 1
plt.subplot(223), plt.imshow(error_image, 'gray'),  plt.title(f'Size outliers')
error_image = np.zeros((TRAY_ROW_COUNT, TRAY_COLUMN_COUNT))
error_image[color_outliers[:, 0], color_outliers[:, 1]] = 1
plt.subplot(224), plt.imshow(error_image, 'gray'),  plt.title(f'Color outliers')

In [ ]:
plt.figure(figsize=(20,20))
for index in range(number_of_images):
    plt.subplot(7,5,1+index), plt.imshow(convertYCbCrtoRGB(seedling_color_model_each_cell_means_buffer[index])), plt.title(image_names_list[index].split(".")[0])

In [ ]:
plt.figure(figsize=(20,20))
for index in range(number_of_images):
    plt.subplot(7,5,1+index), plt.imshow(np.log(ellipse_size_buffer[index]+1), 'jet',  vmin=0, vmax=np.log(ellipse_size_buffer+1).max()), plt.title(image_names_list[index].split(".")[0]), plt.colorbar()

In [ ]:
plt.imshow(seedling_status_confidence_buffer[19], 'terrain'), plt.colorbar()

In [ ]:
plt.imshow(seedling_status_confidence_buffer[19], 'terrain_r', vmin = -0.5, vmax = 1.5)
colorbar = plt.colorbar(fraction=0.05, pad=0.04,)
colorbar.set_ticks([0, 0.5, 1])

In [ ]:

## seedling_status_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)) : สถานะต้นกล้าในแต่ละภาพ (ภาพที่, แถวที่, หลักที่, สถานะ) 0 ไม่มี 1 มี

## seedling_position_model_means_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT, 2)) : mean ตำแหน่งต้นกล้าแต่ละหลุมในแต่ละภาพ (ภาพที่, แถวที่, หลักที่, vector(2))
## seedling_position_model_covariances_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT, 2, 2)) : covariance matrix ตำแหน่งต้นกล้าแต่ละหลุมในแต่ละภาพ (ภาพที่, แถวที่, หลักที่, matrix(2,2))

# number_of_color_models_buffer = np.zeros(number_of_images, dtype=int) : จำนวน cluster ของ color model ในแต่ละภาพ (ภาพที่, จำนวน)

# color_model_means_buffer = np.zeros((number_of_images, MAX_NUM_GAUSSIAN_MODELS, 3)) : mean ของ color model ของต้นกล้าและพื้นหลังในแต่ละภาพ (ภาพที่, โมเดลสีที่, vector(3)) โมเดลสีแรกคือโมเดลสีผัก โมเดลสีที่เกินจากจำนวนโมเดลสีในแต่ละภาพมีค่าเป็น 0
# color_model_covariances_buffer = np.zeros((number_of_images, MAX_NUM_GAUSSIAN_MODELS, 3, 3)) : covariance matrix ของ color model ของต้นกล้าและพื้นหลังในแต่ละภาพ (ภาพที่, โมเดลสีที่, matrix(3,3)) โมเดลสีแรกคือโมเดลสีผัก โมเดลสีที่เกินจากจำนวนโมเดลสีในแต่ละภาพมีค่าเป็น 0

# seedling_color_model_each_cell_means_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT, 3)) : mean ของ color model ของต้นกล้าแต่ละหลุมในแต่ละภาพ (ภาพที่, แถวที่, หลักที่, vector(3)) หลุมที่ไม่มีต้นกล้ามีค่าเป็น 0
# seedling_color_model_each_cell_covariances_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT, 3, 3)) : covariance matrix ของ color model ของต้นกล้าแต่ละหลุมในแต่ละภาพ (ภาพที่, แถวที่, หลักที่, matrix(3,3)) หลุมที่ไม่มีต้นกล้ามีค่าเป็น 0

# ellipse_size_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)) : ขนาดต้นกล้าแต่ละหลุมในแต่ละภาพจากการหาพื้นที่วงรี (ภาพที่, แถวที่, หลักที่, ขนาด) หลุมที่ไม่มีต้นกล้ามีค่าเป็น 0
# seedling_size_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)) : ขนาดต้นกล้าแต่ละหลุมในแต่ละภาพจาก sum probability (ภาพที่, แถวที่, หลักที่, ขนาด)
# seedling_size_to_cell_ratio_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)) : อัตราส่วนขนาดต้นกล้าแต่ละหลุมแต่ขนาดหลุมในแต่ละภาพ (ภาพที่, แถวที่, หลักที่, อัตราส่วน)

# distance_position_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)) : ระยะห่างระหว่างตำแหน่งต้นกล้ากับตำแหน่งจุดศูนย์กลางหลุมปลูก (ภาพที่, แถวที่, หลักที่, ระยะห่าง)
# distance_color_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)) : ระยะห่างระหว่างค่าสีต้นกล้าแต่ละหลุมกับค่าเฉลี่ยค่าสีต้นกล้าของทั้งถาด (ภาพที่, แถวที่, หลักที่, ระยะห่าง)

# seedling_status_confidence_buffer = np.zeros((number_of_images, TRAY_ROW_COUNT, TRAY_COLUMN_COUNT)) : ความเชื่อมั่นที่แต่ละหลุมจะมีต้นกล้า (ภาพที่, แถวที่, หลักที่, ค่าความเชื่อมั่น)

In [ ]:
# status_confidence_over_threshold : สถานะต้นกล้าที่ผ่านเกณฑ์ความเชื่อมั่นที่ (ภาพที่, แถวที่, หลักที่, สถานะ)
# over_threshold_cells : หลุมที่มีต้นกล้า (n, 2)
# under_threshold_cells : หลุมที่ไม่มีต้นกล้า (n, 2)

# size_outliers : หลุมที่มีขนาดผิดปกติ (n, 2)
# mean_size : ค่าเฉลี่ยขนาดต้นกล้าในหน่วยตารางมิลลิเมตร

# color_outliers : หลุมที่มีสีผิดปกติ (n, 2)
# mean_color_rgb = ค่าเฉลี่ยสีต้นกล้า RGB